# Proyecto (Dash ejercicio 2)

## Funciones

In [ ]:
def get_datasets(Country = "Spain", Countries = ["Spain","France","Germany"]):
    # Leer csv
    data = (pd.read_csv("Data_files/covid_Europe.csv"))
    # Crear columna "Date" con formato datetime
    data["Date"] = pd.to_datetime(data["dateRep"], format="%d/%m/%Y")
    # Generar dataset y lista de datasets
    country_data = data[data["countriesAndTerritories"] == Country]
    countries_data = [data[data["countriesAndTerritories"] == _c] for _c in Countries]

    return data, country_data, countries_data


def gen_figs(country, country_data, countries, countries_data):
    # https://plotly.com/python/line-charts/#line-plot-modes
    
    figs = {}
    # Figures of one single country (First section)
    # Figure CasesAndDeaths
    figs["CasesAndDeaths"] = go.Figure()
    figs["CasesAndDeaths"].add_trace(go.Scatter(x=country_data["Date"], y=country_data["cases"]/country_data["cases"].max(),
                        mode='lines',
                        name='Cases'))
    figs["CasesAndDeaths"].add_trace(go.Scatter(x=country_data["Date"], y=country_data["deaths"]/country_data["deaths"].max(),
                        mode='lines',
                        name='Deaths'))
    figs["CasesAndDeaths"].update_layout(
                       xaxis_title='Date',
                       yaxis_title='Normallized to max',
                       title='Visualizando '+country)

    # Figures of one multiple countries (Second section)
    # Figure CasesCountries
    figs["CasesCountries"] = go.Figure()
    N_countries = len(countries)
    for i in range(N_countries):
        tdata = countries_data[i]
        figs["CasesCountries"].add_trace(go.Scatter(x=tdata["Date"], y=tdata["cases"],
                    mode='lines',
                    name=countries[i]))

    return figs

## Mínima app -> Layout básico + Callback básico

In [ ]:
app.layout = html.Div(
    [
        dbc.Row(dbc.Col(html.Img(src="assets/LogoDash.png"), width = 10)),
        dbc.Row(dbc.Col(html.H5(header), width = 10), style = style_1),
        dbc.Row([dbc.Col(html.Button('Actualizar datos', id='update-data', n_clicks=0),
                         style = style_1, width = 1),
                 dbc.Col(html.Div(children="", id='update-date'),
                         style = style_1, width = 9),
                 
                ]),
        dbc.Row(dbc.Col(html.Img(src="assets/LineDash.png"), width = 10)),

        dbc.Row(
            [
                dbc.Col(html.Img(src="assets/panelA.png"),
                        width = 3,style = style_1),
                dbc.Col(dcc.Graph(figure = figs["CasesAndDeaths"], id="F_CD"), width = 7)
            ]
        ),
        dbc.Row(dbc.Col(html.Img(src="assets/LineDash.png"), width = 10)),

        dbc.Row(
            [
                dbc.Col(html.Img(src="assets/panelB.png"), 
                        width = 3,style = style_1),
                dbc.Col(dcc.Graph(figure = figs["CasesCountries"], id="F_CC"), width = 7)
            ]
        ),

        dbc.Row(dbc.Col(html.Img(src="assets/LineDash.png"), width = 10)),

    ]
)

@app.callback(
    Output('update-date', 'children'),
    Output('F_CD', 'figure'),
    Output('F_CC', 'figure'),
    Input('update-data', 'n_clicks'),
)
def update_output(n_clicks):
    
    if n_clicks > 0: 
        url = 'https://opendata.ecdc.europa.eu/covid19/nationalcasedeath_eueea_daily_ei/csv/data.csv'
        download_as(url, name = "covid_Europe.csv")
        text = "Actualizado! -> "+subprocess.run("date", shell=True, capture_output=True, text=True).stdout.strip()
    else:
        text = "Los ficheros de datos no han sido actualizados desde el inicio de la sesión"

    figs = {}
    figs["CasesAndDeaths"] = go.Figure() # Sólo instanciamos
    figs["CasesCountries"] = go.Figure()

    #return text, 0, figs["CasesAndDeaths"], figs["CasesCountries"]
    return text, figs["CasesAndDeaths"], figs["CasesCountries"]


#--------------- Launch/Update the app ------------------------------------------------------------------------------
if __name__ == "__main__":
    app.run_server(debug=True)

## Paneles

In [ ]:
Panel1 = dcc.Dropdown(
    id="region-filter", # Identificador
    options=[{"label": region, "value": region} for region in regions], # Opciones para elegir
    value="Spain", # Valor inicial
    multi=False, # Multiples valores?
    clearable=False,
    className="dropdown",
)

Panel2 = dcc.Dropdown(
    id="countries-filter", # Identificador
    options=[{"label": region, "value": region} for region in regions], # Opciones para elegir
    value=["Spain", "Portugal", "Germany"], # Valor inicial
    multi=False, # Multiples valores?
    clearable=False,
    className="dropdown",
)

Panel3 = dcc.DatePickerRange(
    id='datePicker',
    min_date_allowed=date(2020, 1, 1),
    max_date_allowed=date(2022, 10, 19),
    start_date=date(2020, 1, 1),
    end_date=date(2022, 10, 19),
)

## Otros códigos de interés:

In [ ]:
# Si no es una lista, incluir en lista
if not isinstance(countries, list):
        countries = [countries]

# Selección temporal
if start_date != None and end_date != None:
        country_data = country_data[(country_data['Date'] >= start_date) & (country_data['Date'] <= end_date)]

## Cómo obtener la fecha de actualización de un fichero de datos de una web desde el código

In [9]:
import requests

# Fetch the web page content
url = "https://opendata.ecdc.europa.eu/covid19/nationalcasedeath_eueea_daily_ei/"  # Replace with the actual URL of the web page
webpage = requests.get(url, {'Cache-Control': 'no-cache'})
text = webpage.text

# Split the content into lines
lines = text.split("\n")

# Search for a specific line and get the previous line
search_str = "/csv/"  # Replace with the line you want to search
info_line = ""

for i, line in enumerate(lines):
    if search_str in line:
        info_line = line
        break

#print(info_line)

# Perform the cuts
cut_post = '        &lt;dir&gt; <A HREF="/covid19/nationalcasedeath_eueea_daily_ei/csv'
idx_cut = info_line.find(cut_post)

dateStr =  info_line[idx_cut-len("10/27/2022  9:30 AM"):idx_cut]
print(dateStr)

10/27/2022  9:30 AM
